In [18]:
# from mido import MidiFile
from funcMidi import * # create by SH
MidiFileName = 'p20190612_midi_test_mix.mid'
# MidiFileName = 'p20190716_midi_test.mid'
# MidiFileName = 'p20190818 mix from TMD.mid'
# MidiFileName = 'Fleck, Bela - Daybreak.mid'
# MidiFileName = 'Aguado_12valses_Op1_No1.mid'

# midiArray = MidiCategorize(MidiFileName)
(midiArray, dicEvent) = MidiCategorize(MidiFileName)
dicNoteOnFingerBoard = funcCreatNoteDic()
midiArray

Track 0: Electric Guitar
capo = 0
tuning = [64, 59, 55, 50, 45, 40]
Notes in Finger board:
[64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76]
[59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71]
[55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67]
[50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62]
[45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57]
[40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52]


[[45, 0, 240, 'MEL', 0],
 [53, 240, 480, 'MEL', 1],
 [57, 480, 1440, 'MEL', 2],
 [57, 1920, 2400, 'MIX', 3],
 [40, 1920, 3840, 'MIX', 3],
 [62, 2400, 2880, 'MIX', 3],
 [60, 2880, 3360, 'MIX', 3],
 [55, 3360, 3840, 'MIX', 3],
 [48, 3840, 4800, 'CHO', 4],
 [52, 3840, 4800, 'CHO', 4],
 [60, 3840, 4800, 'CHO', 4],
 [55, 3840, 4800, 'CHO', 4],
 [64, 3840, 4800, 'CHO', 4],
 [50, 4800, 5760, 'CHO', 5],
 [57, 4800, 5760, 'CHO', 5],
 [62, 4800, 5760, 'CHO', 5],
 [66, 4800, 5760, 'CHO', 5],
 [0, 6759, 6760, 'dummy line', 6]]

In [65]:
# midiArray = midiArray[3:9]
midiArray = [ [48, 3840, 4800, 'CHO', 4],
 [52, 3840, 4800, 'CHO', 4],
 [60, 3840, 4800, 'CHO', 4],
 [55, 3840, 4800, 'CHO', 4],
 [64, 3840, 4800, 'CHO', 4],
 [50, 4800, 5760, 'CHO', 5],
 [57, 4800, 5760, 'CHO', 5],
 [62, 4800, 5760, 'CHO', 5],
 [66, 4800, 5760, 'CHO', 5],
 [0, 6759, 6760, 'dummy line', 6]]
midiArray

[[48, 3840, 4800, 'CHO', 4],
 [52, 3840, 4800, 'CHO', 4],
 [60, 3840, 4800, 'CHO', 4],
 [55, 3840, 4800, 'CHO', 4],
 [64, 3840, 4800, 'CHO', 4],
 [50, 4800, 5760, 'CHO', 5],
 [57, 4800, 5760, 'CHO', 5],
 [62, 4800, 5760, 'CHO', 5],
 [66, 4800, 5760, 'CHO', 5],
 [0, 6759, 6760, 'dummy line', 6]]

In [66]:
# list by event
events = []
nn = 0
while nn < len(midiArray):
    if midiArray[nn][3] == 'MEL':
        events.append([midiArray[nn][:-1]])
        nn = nn + 1
    elif midiArray[nn][3] == 'MIX' or midiArray[nn][3] == 'CHO':
        subEvents = []
        currentEvent = midiArray[nn][4]
        while midiArray[nn][4] == currentEvent:
            subEvents.append(midiArray[nn][:-1])
            nn = nn + 1
        events.append(subEvents)
    else:
        break

events
# subEvents

[[[48, 3840, 4800, 'CHO'],
  [52, 3840, 4800, 'CHO'],
  [60, 3840, 4800, 'CHO'],
  [55, 3840, 4800, 'CHO'],
  [64, 3840, 4800, 'CHO']],
 [[50, 4800, 5760, 'CHO'],
  [57, 4800, 5760, 'CHO'],
  [62, 4800, 5760, 'CHO'],
  [66, 4800, 5760, 'CHO']]]

In [67]:
# unique time
allTime = [x[1:3] for x in midiArray[:-1] ] # ignore dummy line
flatAllTime = sum(allTime, [])
uniqTime = list(set(flatAllTime))
uniqTime.sort()

uniqTime

[3840, 4800, 5760]

In [68]:
# 依照時間 產生出note 包含tied note
eventsNotes = []
for event in events:
    if len(event) == 1:
        eventsNotes.append([event[0][0]])
    elif event[0][-1] =='CHO':
        choNotes = [x[0] for x in event]
        eventsNotes.append(choNotes)
    elif event[0][-1] == 'MIX':
        allTime = [x[1:3] for x in event]
        flatAllTime = sum(allTime, [])
        uniqTime = list(set(flatAllTime))
        uniqTime.sort()

        subEventNotes = []
        tempNotes = []
        for time in uniqTime[:-1]:

            for note in event:
                if note[1] < time < note[2]:
                    try: 
                        tempNotes[tempNotes.index(note[0])] = -note[0]
                    except:
                        pass
                elif note[1] == time:
                    tempNotes.append(note[0])
                elif note[2] == time:
                    try:
                        tempNotes.remove(note[0])
                    except:
                        tempNotes.remove(-note[0])    
            subEventNotes.append(copy.deepcopy(tempNotes)) # should use deepcopy instead of copy
        for x in subEventNotes:
            eventsNotes.append(x) 
        
eventsNotes

[[48, 52, 60, 55, 64], [50, 57, 62, 66]]

In [104]:
# debug, Chord cost
debugNote = [48, 52, 60, 55, 64]
for costChordFinger4 in range(2,100, 20):
    subPossible = funcChordSolution(list(map(abs, debugNote)), dicNoteOnFingerBoard)
    for possible in subPossible:
        cost = funcCalChordCost(possible)
        print(str(cost) + str(possible))
    print()

16.6[[6, 8, 3], [5, 7, 2], [3, 5, 1], [4, 5, 1], [1, 0, 0]]
11.6[[6, 8, 3], [5, 7, 2], [3, 5, 1], [4, 5, 1], [2, 5, 1]]
9.6[[5, 3, 4], [4, 2, 2], [2, 1, 1], [3, 0, 0], [1, 0, 0]]
9.6[[5, 3, 4], [4, 2, 3], [2, 1, 1], [3, 0, 0], [1, 0, 0]]
9.6[[5, 3, 4], [4, 2, 3], [2, 1, 2], [3, 0, 0], [1, 0, 0]]
18.0[[6, 8, 2], [5, 7, 1], [4, 10, 4], [3, 0, 0], [1, 0, 0]]
8.6[[5, 3, 3], [4, 2, 2], [2, 1, 1], [3, 0, 0], [1, 0, 0]]
17.6[[6, 8, 4], [5, 7, 3], [3, 5, 1], [4, 5, 1], [1, 0, 0]]
12.6[[6, 8, 4], [5, 7, 3], [3, 5, 1], [4, 5, 1], [2, 5, 1]]
17.6[[6, 8, 4], [5, 7, 2], [3, 5, 1], [4, 5, 1], [1, 0, 0]]
12.6[[6, 8, 4], [5, 7, 2], [3, 5, 1], [4, 5, 1], [2, 5, 1]]

16.6[[6, 8, 3], [5, 7, 2], [3, 5, 1], [4, 5, 1], [1, 0, 0]]
11.6[[6, 8, 3], [5, 7, 2], [3, 5, 1], [4, 5, 1], [2, 5, 1]]
9.6[[5, 3, 4], [4, 2, 2], [2, 1, 1], [3, 0, 0], [1, 0, 0]]
9.6[[5, 3, 4], [4, 2, 3], [2, 1, 1], [3, 0, 0], [1, 0, 0]]
9.6[[5, 3, 4], [4, 2, 3], [2, 1, 2], [3, 0, 0], [1, 0, 0]]
18.0[[6, 8, 2], [5, 7, 1], [4, 10, 4], [3, 0,

In [77]:
# run through events
# 有BUG存在!!!
# 目前len = 1的情況下 沒有去判斷延音

possible =[]
for choNote in eventsNotes:
    if len(choNote) == 1 :
        # BUG IS HERE ---------------------------------vvvvvvvvvvvvvv
        if choNote[0] < 0:
            print("BUG, no constraints for tied note when there is only one note")
        possible.append([ [x] for x in funcNote2Domain(abs(choNote[0]), dicNoteOnFingerBoard)])
#         possible.append([funcNote2Domain(choNote[0], dicNoteOnFingerBoard)])
    else:
        subPossible = funcChordSolution(list(map(abs, choNote)), dicNoteOnFingerBoard)
        possible.append(subPossible)
possible

# 如果是從前一個就start, 還沒結束的NOte, 改為 -Note (如-55)
# 當作一個標記flag
# 算chord solution時要拿abs
# 可是算path的時候
# 有看到負號就只能抓前一組相同的note posibtion


[[[[6, 8, 3], [5, 7, 2], [3, 5, 1], [4, 5, 1], [1, 0, 0]],
  [[6, 8, 3], [5, 7, 2], [3, 5, 1], [4, 5, 1], [2, 5, 1]],
  [[5, 3, 4], [4, 2, 2], [2, 1, 1], [3, 0, 0], [1, 0, 0]],
  [[5, 3, 4], [4, 2, 3], [2, 1, 1], [3, 0, 0], [1, 0, 0]],
  [[5, 3, 4], [4, 2, 3], [2, 1, 2], [3, 0, 0], [1, 0, 0]],
  [[6, 8, 2], [5, 7, 1], [4, 10, 4], [3, 0, 0], [1, 0, 0]],
  [[5, 3, 3], [4, 2, 2], [2, 1, 1], [3, 0, 0], [1, 0, 0]],
  [[6, 8, 4], [5, 7, 3], [3, 5, 1], [4, 5, 1], [1, 0, 0]],
  [[6, 8, 4], [5, 7, 3], [3, 5, 1], [4, 5, 1], [2, 5, 1]],
  [[6, 8, 4], [5, 7, 2], [3, 5, 1], [4, 5, 1], [1, 0, 0]],
  [[6, 8, 4], [5, 7, 2], [3, 5, 1], [4, 5, 1], [2, 5, 1]]],
 [[[6, 10, 1], [5, 12, 3], [4, 12, 4], [3, 11, 2]],
  [[4, 0, 0], [3, 2, 1], [2, 3, 2], [1, 2, 1]],
  [[4, 0, 0], [3, 2, 1], [2, 3, 3], [1, 2, 1]],
  [[4, 0, 0], [3, 2, 1], [2, 3, 3], [1, 2, 2]],
  [[4, 0, 0], [3, 2, 1], [2, 3, 4], [1, 2, 1]],
  [[4, 0, 0], [3, 2, 1], [2, 3, 4], [1, 2, 2]],
  [[4, 0, 0], [3, 2, 1], [2, 3, 4], [1, 2, 3]],
  [[4, 0,

In [84]:
# del possible[1][7:]
possible

[[[[5, 3, 4], [4, 2, 2], [2, 1, 1], [3, 0, 0], [1, 0, 0]],
  [[5, 3, 4], [4, 2, 3], [2, 1, 1], [3, 0, 0], [1, 0, 0]],
  [[5, 3, 4], [4, 2, 3], [2, 1, 2], [3, 0, 0], [1, 0, 0]],
  [[5, 3, 3], [4, 2, 2], [2, 1, 1], [3, 0, 0], [1, 0, 0]]],
 [[[4, 0, 0], [3, 2, 1], [2, 3, 2], [1, 2, 1]],
  [[4, 0, 0], [3, 2, 1], [2, 3, 3], [1, 2, 1]],
  [[4, 0, 0], [3, 2, 1], [2, 3, 3], [1, 2, 2]],
  [[4, 0, 0], [3, 2, 1], [2, 3, 4], [1, 2, 1]],
  [[4, 0, 0], [3, 2, 1], [2, 3, 4], [1, 2, 2]],
  [[4, 0, 0], [3, 2, 1], [2, 3, 4], [1, 2, 3]],
  [[4, 0, 0], [3, 2, 2], [2, 3, 4], [1, 2, 3]]]]

In [96]:
# @@@@@@@@@ MIX 的版本 @@@@@@@@@
# 從MELODY 摳過來的
# 這邊不能合成function
# 因為要跟MEL 和 CHO 一起混著做
# 這邊只是確定funcCalMixCostMatrix 正確
costBest = []

iniSize = len(possible[0])
npPreMinValue = np.zeros(iniSize)
preMinIndex = [[0] for i in range(iniSize)]
# for i in range(2,3):
# for i in range(1):
for i in range(len(possible)-1):
    event0Possible = possible[i]
    event1Possible = possible[i+1]
    mixNotes0 = eventsNotes[i]
    mixNotes1 = eventsNotes[i+1]
    # calculate MIX cost
    costMatrixUnitTran, costMatrixUnitChord = funcCalTranCostMatrix(event0Possible, event1Possible, mixNotes0, mixNotes1)    
    npCostMatrixUnitTran = np.array(costMatrixUnitTran)
    npCostMatrixUnitChord = np.array(costMatrixUnitChord)
    npCostMatrixUnit = npCostMatrixUnitChord + npCostMatrixUnitTran
    
    # print each costMatrixUnit independently, for debug
    print(np.around(npCostMatrixUnitChord, decimals = 0))
    print("")
    print(np.around(npCostMatrixUnitTran, decimals = 0))
    print("")
    print(np.around(npCostMatrixUnit, decimals = 0))
    # do matrix summary: minValue + minIndex
    npMinValue = np.min(npCostMatrixUnit, axis = 0)    
    npMinIndex = np.argmin(npCostMatrixUnit, axis = 0)
    minIndex = list(npMinIndex)
    
    # initialize for combining with previous event
    npCombMinValue = np.zeros(len(npMinValue))
    combMinIndex = [0 for icmi in range(len(npMinValue))]

    # minValue 
    for imin in range(len(npMinValue)):
        npCombMinValue[imin] = npMinValue[imin] + npPreMinValue[npMinIndex[imin]]
        combMinIndex[imin] = copy.deepcopy(preMinIndex[npMinIndex[imin]])
        combMinIndex[imin].append(copy.deepcopy(minIndex[imin]))
    npPreMinValue = npCombMinValue
    preMinIndex = combMinIndex

#     minSummary = np.vstack
    costBest.append(copy.deepcopy([npCombMinValue,combMinIndex]))
# costBest[-1]

[[58. 58. 52. 61. 55. 55. 55.]
 [58. 58. 52. 61. 55. 55. 55.]
 [58. 58. 52. 61. 55. 55. 55.]
 [55. 55. 49. 58. 52. 52. 52.]]

[[65. 53. 50. 63. 60. 53. 50.]
 [62. 64. 53. 63. 52. 59. 48.]
 [54. 56. 60. 52. 56. 52. 55.]
 [63. 59. 55. 55. 51. 54. 51.]]

[[123. 111. 102. 124. 115. 107. 105.]
 [119. 122. 105. 124. 107. 114. 103.]
 [111. 114. 111. 113. 110. 106. 110.]
 [118. 114. 104. 112. 103. 106. 103.]]


In [100]:
event1Possible
costChordWeight = 3
costChordFinger4 =2 

In [101]:
simple()

2


In [98]:
def simple():
    print(costChordFinger4)

In [75]:
# debug use, 計算特定chord之間的cost
i = 2
event0Possible = possible[i]
event1Possible = possible[i+1]
mixNotes0 = eventsNotes[i]
mixNotes1 = eventsNotes[i+1]
mixPos0 = event0Possible[0]
mixPos1 = event1Possible[1]

print(mixPos0)
print(mixPos1)
print(mixNotes0)
print(mixNotes1)
funcCalMixCostUnit(mixPos0, mixPos1,mixNotes0, mixNotes1)

[[3, 2, 1]]
[[3, 2, 1], [6, 0, 0]]
[57]
[57, 40]


(4.6000000000000005, 2.9000000000000004)

In [50]:
# 這應該算是funcCalMixCostUnit
# 因為這只算了一個unit
# 真的funCalMixCost 需要產出costMatrixUnit, 這個unit是event event之間的unit

# 這個Function 也可以用在melody to Mix, 不會被限制
# 他也許可以改為funcTranCostUnit, 專門比較transition的cost
def funcCalTranCostUnit(mixPos0, mixPos1,mixNotes0, mixNotes1):

    costTran = 0
    for ipos0 in range(len(mixPos0)):
#         print("ipos0=" + str(ipos0))
        for ipos1 in range(len(mixPos1)):
#             print("ipos1=" + str(ipos1))
            if mixNotes1[ipos1] < 0:
                try:
                    relativeNote0Index = mixNotes0.index(abs(mixNotes1[ipos1]))
                except:
                    relativeNote0Index = mixNotes0.index(   (mixNotes1[ipos1]))
#                 print("relativeNote0Index = " + str(relativeNote0Index))
                if mixPos1[ipos1] != mixPos0[ relativeNote0Index]:
#                     print("negative and !=")
                    costMixTotal = float("inf")
                    break
                # 可以走到這代表前次和這次的position相同
                # costAlong = 0, 所以不用做任何相加
            else:
#                 print("pass")
                costAlong    = funcCalCostAlong(mixPos0[ipos0], mixPos1[ipos1])
                costAcross   = funcCalCostAcross(mixPos0[ipos0], mixPos1[ipos1])

#                 print(mixPos0[ipos0])
                costTran = costTran + costAlong + costAcross
    ##### debug use #####
#     if mixPos0 == [[6, 3, 1], [2, 0, 0], [3, 0, 0]] and mixPos1 ==[[6, 3, 1], [4, 0, 0]] :
        
#                 print("costAlong = " + str(costAlong))
#                 print("costAcross = " + str(costAcross))
#                 print("costMixTotal = " + str(costMixTotal))
    costChord0   = funcCalChordCost(mixPos0)
    costChord1   = funcCalChordCost(mixPos1)
    costChord = (costChord0 + costChord1)* costChordWeight
#     costMixTotal = costMixTotal+ costChord
#     print(costChord)
    return costTran, costChord
        
# costMixTotal

In [58]:
# Calculate MIX transition cost

def funcCalTranCostMatrix(event0Possible, event1Possible, mixNotes0, mixNotes1):
    costMatrixUnitTran = []
    costMatrixUnitChord = []
    for mixPos0 in event0Possible:
        pos0beg =[]
        pos0begChord = []
        for mixPos1 in event1Possible:

            # cost is putting here
            costTran, costChord = funcCalTranCostUnit(mixPos0, mixPos1,mixNotes0, mixNotes1)
            
#             ######### debug #########
#             if mixPos0 == [[6, 3, 1], [2, 0, 0], [3, 0, 0]] and mixPos1 ==[[6, 3, 1], [4, 0, 0]] :
#                 costMixTotal, costChord = funcCalMixCostUnit(mixPos0, mixPos1,mixNotes0, mixNotes1)

            pos0beg.append(costTran)
            pos0begChord.append(costChord)
        costMatrixUnitTran.append(pos0beg)
        costMatrixUnitChord.append(pos0begChord)
    return costMatrixUnitTran, costMatrixUnitChord

In [73]:
onlyBest = True
Solution = []

# if onlyBest:
# chooses = np.argmin(costBest[-1][0], axis = 0)
# else:
#     chooses = range(len(costBest[-1][1]))
for choose in range(len(costBest[-1][1])):
# for choose in list(chooses):

    bestPathNumber = costBest[-1][1][choose][1:]
    print(bestPathNumber)
    bestPath = []
    for i in range(len(bestPathNumber)):
        bestPath.append(possible[i][bestPathNumber[i]])
    bestPath.append(possible[-1][choose])
    #     print("cost=" + str(costBest[-1][0][choose]) +" " + str(bestPath))
    print("cost=" + '%.2f' % costBest[-1][0][choose] +" " + str(bestPath))
    Solution.append(["cost =", costBest[-1][0][choose], bestPath ])


[6]
cost=146.54 [[[5, 3, 3], [4, 2, 2], [2, 1, 1], [3, 0, 0], [1, 0, 0]], [[6, 10, 1], [5, 12, 3], [4, 12, 4], [3, 11, 2]]]
[4]
cost=63.40 [[[5, 3, 4], [4, 2, 3], [2, 1, 2], [3, 0, 0], [1, 0, 0]], [[4, 0, 0], [3, 2, 1], [2, 3, 2], [1, 2, 1]]]
[2]
cost=62.65 [[[5, 3, 4], [4, 2, 2], [2, 1, 1], [3, 0, 0], [1, 0, 0]], [[4, 0, 0], [3, 2, 1], [2, 3, 3], [1, 2, 1]]]
[2]
cost=58.65 [[[5, 3, 4], [4, 2, 2], [2, 1, 1], [3, 0, 0], [1, 0, 0]], [[4, 0, 0], [3, 2, 1], [2, 3, 3], [1, 2, 2]]]
[4]
cost=62.15 [[[5, 3, 4], [4, 2, 3], [2, 1, 2], [3, 0, 0], [1, 0, 0]], [[4, 0, 0], [3, 2, 1], [2, 3, 4], [1, 2, 1]]]
[6]
cost=59.52 [[[5, 3, 3], [4, 2, 2], [2, 1, 1], [3, 0, 0], [1, 0, 0]], [[4, 0, 0], [3, 2, 1], [2, 3, 4], [1, 2, 2]]]
[4]
cost=60.90 [[[5, 3, 4], [4, 2, 3], [2, 1, 2], [3, 0, 0], [1, 0, 0]], [[4, 0, 0], [3, 2, 1], [2, 3, 4], [1, 2, 3]]]
[3]
cost=57.52 [[[5, 3, 4], [4, 2, 3], [2, 1, 1], [3, 0, 0], [1, 0, 0]], [[4, 0, 0], [3, 2, 2], [2, 3, 4], [1, 2, 3]]]
[5]
cost=117.50 [[[6, 8, 2], [5, 7, 1], [4,

In [60]:
# finalDecision = costBest[-1]
# choose = np.argmin(costBest[-1][0], axis = 0)
# finalDecision
Solution

[['cost =',
  184.5361111111111,
  [[[5, 0, 0]],
   [[4, 3, 3]],
   [[3, 2, 2]],
   [[3, 2, 2], [6, 0, 0]],
   [[6, 0, 0], [2, 3, 3]],
   [[6, 0, 0], [2, 1, 1]],
   [[6, 0, 0], [3, 0, 0]],
   [[5, 3, 3], [4, 2, 2], [2, 1, 1], [3, 0, 0], [1, 0, 0]],
   [[6, 10, 1], [5, 12, 3], [4, 12, 4], [3, 11, 2]]]],
 ['cost =',
  101.9,
  [[[5, 0, 0]],
   [[4, 3, 3]],
   [[3, 2, 2]],
   [[3, 2, 2], [6, 0, 0]],
   [[6, 0, 0], [2, 3, 3]],
   [[6, 0, 0], [2, 1, 1]],
   [[6, 0, 0], [3, 0, 0]],
   [[5, 3, 4], [4, 2, 3], [2, 1, 2], [3, 0, 0], [1, 0, 0]],
   [[4, 0, 0], [3, 2, 1], [2, 3, 2], [1, 2, 1]]]],
 ['cost =',
  101.15,
  [[[5, 0, 0]],
   [[4, 3, 3]],
   [[3, 2, 2]],
   [[3, 2, 2], [6, 0, 0]],
   [[6, 0, 0], [2, 3, 3]],
   [[6, 0, 0], [2, 1, 1]],
   [[6, 0, 0], [3, 0, 0]],
   [[5, 3, 4], [4, 2, 2], [2, 1, 1], [3, 0, 0], [1, 0, 0]],
   [[4, 0, 0], [3, 2, 1], [2, 3, 3], [1, 2, 1]]]],
 ['cost =',
  97.15,
  [[[5, 0, 0]],
   [[4, 3, 3]],
   [[3, 2, 2]],
   [[3, 2, 2], [6, 0, 0]],
   [[6, 0, 0], [2, 3, 3